In [2]:
from google.colab import files
uploaded = files.upload()


Saving Quebec Production Report_Chennai.xlsx to Quebec Production Report_Chennai.xlsx
Saving TDM Hourly Production Report.xlsx to TDM Hourly Production Report (1).xlsx


In [3]:
!pip install openpyxl scikit-learn pandas matplotlib

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


In [6]:
# Load both Excel sheets
df1 = pd.read_excel("Quebec Production Report_Chennai.xlsx")
df2 = pd.read_excel("TDM Hourly Production Report.xlsx")

print("Quebec Report Shape:", df1.shape)
print("TDM Report Shape:", df2.shape)

# Display first few rows
df1.head(), df2.head()


Quebec Report Shape: (515, 18)
TDM Report Shape: (633, 27)


(                  Date         Name Project  Target  Achieved  Deviation  \
 0  2025-01-08 00:00:00   Soundariya  Quebec  7500.0       0.0     7500.0   
 1  2025-01-08 00:00:00      Parkavi  Quebec  7500.0       0.0     7500.0   
 2  2025-01-08 00:00:00  Keerthana S  Quebec  7500.0       0.0     7500.0   
 3  2025-01-08 00:00:00      Sowmiya  Quebec  5000.0       0.0     5000.0   
 4  2025-01-08 00:00:00     Ebinesia  Quebec  1500.0     800.0      700.0   
 
           % Reason 2024-09-10 00:00:00  10.10-11.00  2024-11-12 00:00:00  \
 0  0.000000     TL               Final          NaN                  NaN   
 1  0.000000    NaN        Rework Files          NaN                  NaN   
 2  0.000000    NaN        Rework Files          NaN                  NaN   
 3  0.000000    NaN        Rework Files          NaN                  NaN   
 4  0.533333    NaN                 100        100.0                100.0   
 
   2024-12-01 00:00:00 1.30-2.30 2.30-3.20 3.30-4.30  4.30-5.30  5.30-7.

In [10]:
# Top and Bottom 5 performers from Quebec file
top5_df1 = df1.sort_values(by='Achieved', ascending=False).head(5)
bottom5_df1 = df1.sort_values(by='Achieved', ascending=True).head(5)

# Top and Bottom 5 performers from TDM file
top5_df2 = df2.sort_values(by='Achieved', ascending=False).head(5)
bottom5_df2 = df2.sort_values(by='Achieved', ascending=True).head(5)

print("\nTop 5 Performers - Quebec Report:")
display(top5_df1[['Name','Project','Achieved','Target','%']])

print("\nBottom 5 Performers - Quebec Report:")
display(bottom5_df1[['Name','Project','Achieved','Target','%']])

print("\nTop 5 Performers - TDM Report:")
display(top5_df2[['Name','Project','Achieved','Target','%']])

print("\nBottom 5 Performers - TDM Report:")
display(bottom5_df2[['Name','Project','Achieved','Target','%']])



Top 5 Performers - Quebec Report:


,Name,Project,Achieved,Target,%
473,Karthiga,Quebec,40321.0,10000.0,4.0321
490,Karthiga,Quebec,39000.0,10000.0,3.9000
403,Karthiga,Quebec,38118.0,10000.0,3.8118
456,Karthiga,Quebec,37891.0,10000.0,3.7891
506,Pushpa,Quebec,29741.0,10000.0,2.9741



Bottom 5 Performers - Quebec Report:


,Name,Project,Achieved,Target,%
0,Soundariya,Quebec,0.0,7500.0,0.0
42,Parkavi,Quebec,0.0,7500.0,0.0
44,Sowmiya,Quebec,0.0,5000.0,0.0
43,Keerthana S,Quebec,0.0,7500.0,0.0
441,Kausalya,Quebec,0.0,500.0,0.0



Top 5 Performers - TDM Report:


,Name,Project,Achieved,Target,%
393,Vanitha E,HCR,15802.0,15000.0,1.053467
629,Vanitha E,HCR,15037.0,15000.0,1.002467
597,Vanitha E,HCR,14490.0,15000.0,0.966000
501,Vanitha E,HCR,14156.0,15000.0,0.943733
578,Vanitha E,HCR,14123.0,15000.0,0.941533



Bottom 5 Performers - TDM Report:


,Name,Project,Achieved,Target,%
56,Mahalakshmi T,Calabria Italian,0.0,100.0,0.0
335,Suriyakala,Proofing,0.0,30.0,0.0
339,Aswini G,Calabria Italian,0.0,200.0,0.0
341,Sudha S,Calabria Italian,0.0,200.0,0.0
348,Yuvarani Pg,Proofing,0.0,30.0,0.0


For accuracy


In [12]:
!pip install scikit-learn openpyxl pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Load the two Excel files
df1 = pd.read_excel("Quebec Production Report_Chennai.xlsx")
df2 = pd.read_excel("TDM Hourly Production Report.xlsx")

# Convert Date columns to datetime
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

# Function to train model & print accuracy + top/bottom 5
def analyze_file(df, name):
    print(f"\n===== {name} =====")
    # Drop rows with missing Target or Achieved
    df = df.dropna(subset=['Target','Achieved'])
    # Feature & Target
    X = df[['Target']]   # Features
    y = df['Achieved']   # Target variable

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Accuracy metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print("Model R² Accuracy:", round(r2,3))
    print("Mean Absolute Error:", round(mae,3))

    # Add predictions to the dataframe
    df = df.copy()
    df['Predicted'] = model.predict(df[['Target']])
    df['Gap'] = df['Achieved'] - df['Predicted']

    # Top & Bottom 5 performers
    top5 = df.sort_values(by='Achieved', ascending=False).head(5)
    bottom5 = df.sort_values(by='Achieved', ascending=True).head(5)

    print("\nTop 5 Performers:")
    display(top5[['Name','Project','Target','Achieved','Predicted','Gap','%']])

    print("\nBottom 5 Performers:")
    display(bottom5[['Name','Project','Target','Achieved','Predicted','Gap','%']])

# Analyze each file separately
analyze_file(df1, "Quebec Production Report")
analyze_file(df2, "TDM Hourly Production Report")



===== Quebec Production Report =====
Model R² Accuracy: 0.17
Mean Absolute Error: 1864.395

Top 5 Performers:


,Name,Project,Target,Achieved,Predicted,Gap,%
473,Karthiga,Quebec,10000.0,40321.0,6616.774482,33704.225518,4.0321
490,Karthiga,Quebec,10000.0,39000.0,6616.774482,32383.225518,3.9000
403,Karthiga,Quebec,10000.0,38118.0,6616.774482,31501.225518,3.8118
456,Karthiga,Quebec,10000.0,37891.0,6616.774482,31274.225518,3.7891
506,Pushpa,Quebec,10000.0,29741.0,6616.774482,23124.225518,2.9741



Bottom 5 Performers:


,Name,Project,Target,Achieved,Predicted,Gap,%
0,Soundariya,Quebec,7500.0,0.0,4882.854926,-4882.854926,0.0
42,Parkavi,Quebec,7500.0,0.0,4882.854926,-4882.854926,0.0
44,Sowmiya,Quebec,5000.0,0.0,3148.935369,-3148.935369,0.0
43,Keerthana S,Quebec,7500.0,0.0,4882.854926,-4882.854926,0.0
441,Kausalya,Quebec,500.0,0.0,27.880168,-27.880168,0.0



===== TDM Hourly Production Report =====
Model R² Accuracy: 0.998
Mean Absolute Error: 50.467

Top 5 Performers:


,Name,Project,Target,Achieved,Predicted,Gap,%
393,Vanitha E,HCR,15000.0,15802.0,13821.357513,1980.642487,1.053467
629,Vanitha E,HCR,15000.0,15037.0,13821.357513,1215.642487,1.002467
597,Vanitha E,HCR,15000.0,14490.0,13821.357513,668.642487,0.966000
501,Vanitha E,HCR,15000.0,14156.0,13821.357513,334.642487,0.943733
578,Vanitha E,HCR,15000.0,14123.0,13821.357513,301.642487,0.941533



Bottom 5 Performers:


,Name,Project,Target,Achieved,Predicted,Gap,%
56,Mahalakshmi T,Calabria Italian,100.0,0.0,53.769966,-53.769966,0.0
335,Suriyakala,Proofing,30.0,0.0,-10.909975,10.909975,0.0
339,Aswini G,Calabria Italian,200.0,0.0,146.169883,-146.169883,0.0
341,Sudha S,Calabria Italian,200.0,0.0,146.169883,-146.169883,0.0
348,Yuvarani Pg,Proofing,30.0,0.0,-10.909975,10.909975,0.0


In [14]:
!pip install scikit-learn openpyxl pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from google.colab import files

# 1. Upload your files
print("Upload your two Excel files…")
uploaded = files.upload()

# 2. Load the two Excel files
df1 = pd.read_excel("Quebec Production Report_Chennai.xlsx")
df2 = pd.read_excel("TDM Hourly Production Report.xlsx")

# 3. Convert Date columns to datetime
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

# 4. Function to train model, show accuracy & return df with predictions
def analyze_file(df):
    # Drop rows with missing Target or Achieved
    df = df.dropna(subset=['Target','Achieved'])
    # Feature & Target
    X = df[['Target']]   # Features
    y = df['Achieved']   # Target variable

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Accuracy metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print("Model R² Accuracy:", round(r2,3))
    print("Mean Absolute Error:", round(mae,3))

    # Add predictions to the dataframe
    df = df.copy()
    df['Predicted'] = model.predict(df[['Target']])
    df['Gap'] = df['Achieved'] - df['Predicted']
    return df

print("\n===== Quebec Production Report =====")
df1_model = analyze_file(df1)

print("\n===== TDM Hourly Production Report =====")
df2_model = analyze_file(df2)

# 5. Build Top & Bottom 5 tables from each file with predictions
def top_bottom_tables(df):
    top5 = df.sort_values(by='Achieved', ascending=False).head(5)
    bottom5 = df.sort_values(by='Achieved', ascending=True).head(5)
    return top5, bottom5

top5_df1, bottom5_df1 = top_bottom_tables(df1_model)
top5_df2, bottom5_df2 = top_bottom_tables(df2_model)

# 6. Export all four tables to one Excel file
output_file = "Production_Report_TopBottom5.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    top5_df1[['Name','Project','Target','Achieved','Predicted','Gap','%']].to_excel(writer, sheet_name='Quebec_Top5', index=False)
    bottom5_df1[['Name','Project','Target','Achieved','Predicted','Gap','%']].to_excel(writer, sheet_name='Quebec_Bottom5', index=False)
    top5_df2[['Name','Project','Target','Achieved','Predicted','Gap','%']].to_excel(writer, sheet_name='TDM_Top5', index=False)
    bottom5_df2[['Name','Project','Target','Achieved','Predicted','Gap','%']].to_excel(writer, sheet_name='TDM_Bottom5', index=False)

print("\n✅ Exported all Top/Bottom 5 tables to", output_file)

# 7. Download the Excel file to your PC
files.download(output_file)


Upload your two Excel files…


Saving Quebec Production Report_Chennai.xlsx to Quebec Production Report_Chennai (1).xlsx
Saving TDM Hourly Production Report.xlsx to TDM Hourly Production Report (2).xlsx

===== Quebec Production Report =====
Model R² Accuracy: 0.17
Mean Absolute Error: 1864.395

===== TDM Hourly Production Report =====
Model R² Accuracy: 0.998
Mean Absolute Error: 50.467

✅ Exported all Top/Bottom 5 tables to Production_Report_TopBottom5.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>